In [66]:
def unir_diccionarios(a_unir: list[dict[str,str]]) -> dict[str,list[str]]:
    
    res: dict[str,list[str]] = {}
    
    for diccionario in a_unir:
        for clave, valor in diccionario.items():
            if clave in res:
                res[clave].append(valor)
            else:
                res[clave] = [valor]

    return res

In [67]:
unir_diccionarios([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}, {'a': 5, 'c':10}])

{'a': [1, 5], 'b': [2, 3], 'c': [4, 10]}

In [68]:
from queue import Queue
from typing import Union

def procesamiento_pedidos(pedidos: Queue[dict[str, Union[int, str, dict[str, int]]]],
                          stock_productos: dict[str, int],
                          precios_productos: dict[str, float]) -> list[dict[str, Union[int, str, float, dict[str, int]]]]:
  
    # inicializo mis pedidos procesados
    pedidos_procesados: Queue = []
    # llamo mi stock de productos para ir actualizandose segun pasan los pedidos
    stock_productos_final: dict[str, int] = stock_productos

    # ITERACION SOBRE LOS PEDIDOS #
    while not pedidos.empty():

        pedido: dict[str, Union[int, str, dict[str, int]]] = pedidos.get() # tomo primer pedido de la cola

        pedido_procesado: dict[str, Union[int, str, dict[str, int]]] = pedido  # parto del pedido y lo voy procesando
        estado: str = 'completo'  # inicializo el estado, por default 'completo'
        precio_total: float = 0  # inicializo el precio_total

        # Itero los productos y su cantidad pedida #
        for producto_pedido, cantidad_pedida in pedido['productos'].items():

            cantidad_pedida_final: int = 0  # inicializo la cantidad total que se lleva el cliente

            stock: float = stock_productos[producto_pedido]  # tomo el stock del producto

            # Comparo stock con la cantidad pedida #
            if stock == 0: # Sin stock

                estado = 'incompleto'
                pedido_procesado['productos'][producto_pedido] = stock  # !

            elif cantidad_pedida > stock: # Pedido excede stock

                estado = 'incompleto'
                precio_total += ( stock * precios_productos[producto_pedido] )
                cantidad_pedida_final = stock
                pedido_procesado['productos'][producto_pedido] = cantidad_pedida_final  # !

            else: # Pedido <= stock

                precio_total += ( cantidad_pedida * precios_productos[producto_pedido] )
                cantidad_pedida_final = cantidad_pedida
                pedido_procesado['productos'][producto_pedido] = cantidad_pedida_final  # !
            
            # ! : detallo la cantidad final que se lleva el cliente de un pedido

            stock_productos_final[producto_pedido] = stock - cantidad_pedida_final  # Actualizo el stock
        
        pedido_procesado['precio_total'] = precio_total  
        pedido_procesado['estado'] = estado

        pedidos_procesados.append(pedido_procesado)  # agrego el pedido procesado

    return pedidos_procesados

In [92]:
pedidos = Queue()
pedidos.put({'id': 1, 'cliente': 'Juan', 'productos': {'Manzana': 2, 'Pan': 4, 'Factura': 6}})
pedidos.put({'id': 21, 'cliente': 'Gabirela', 'productos': {'Manzana': 2}})
stock_productos = {'Manzana': 3, 'Leche': 5, 'Pan': 4, 'Factura': 10 }
precios_productos = {'Manzana': 3.5, 'Leche': 5.5, 'Pan': 3.5, 'Factura': 2.75 }

In [93]:
procesamiento_pedidos(pedidos, stock_productos, precios_productos)

[{'id': 1,
  'cliente': 'Juan',
  'productos': {'Manzana': 2, 'Pan': 4, 'Factura': 6},
  'precio_total': 37.5,
  'estado': 'completo'},
 {'id': 21,
  'cliente': 'Gabirela',
  'productos': {'Manzana': 1},
  'precio_total': 3.5,
  'estado': 'incompleto'}]